# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 100)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 100:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.


Moe_Szyslak: Ah, isn't that nice. Now, there is a politician

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
from collections import Counter
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function for creating lookup tables
    # Sort the vocabulary by the number of times the word occurs
    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    
    # Dictionary key: integer, value: word
    int_to_vocab = {}
    for i in range(len(sorted_vocab)):
        int_to_vocab[i] = sorted_vocab[i]
    
    # Dictionary key: word, value: integer
    vocab_to_int = {}
    int_to_vocab_items = int_to_vocab.items()
    for i in range(len(int_to_vocab_items)):
        vocab_to_int[int_to_vocab[i]] = i
        
    
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    tokenize_punctuation_dictionary = {'.':'||Period||',
            ',':'||Comma||',
            '"':'||Quotation_Mark||',
            ';':'||Semicolon||', 
            '!':'||Exclamation_Mark||',
            '?':'||Question_Mark||', 
            '(':'||Left_Parentheses||', 
            ')':'||Right_Parentheses||',
            '--':'||Dash||',
            '\n':'||Return||'}
    return tokenize_punctuation_dictionary

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    return inputs, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    
    # Your basic LSTM cell  
    # lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)  
    # Stack up multiple LSTM layers, for deep learning
    # lstm_layers = 2
    # cell = tf.contrib.rnn.MultiRNNCell([lstm] * lstm_layers)
    
    # The code above gives an error Trying to share variable rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel, but specified shape (1024, 2048) and found shape (513, 2048). 
    # Solution https://stackoverflow.com/questions/44615147/valueerror-trying-to-share-variable-rnn-multi-rnn-cell-cell-0-basic-lstm-cell-k
    
    lstm = []
    lstm_layers = 2
    for iiLyr in range(lstm_layers):
        lstm.append(tf.nn.rnn_cell.BasicLSTMCell(rnn_size)) 
    cell = tf.nn.rnn_cell.MultiRNNCell(cells=lstm)
    
    # Getting an initial state of all zeros
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), name='initial_state')   
  
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))  
    embed = tf.nn.embedding_lookup(embedding, input_data)  
    
    #embed = tf.contrib.layers.embed_sequence(input_data, vocab_size, embed_dim)

    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)  
    final_state = tf.identity(final_state, name='final_state')
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    embed = get_embed(input_data, vocab_size, embed_dim)
    (outputs, final_state) = build_rnn(cell, embed)
    predictions = tf.layers.dense(outputs, vocab_size)
    return (predictions, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    
    num_batches = len(int_text)//(batch_size*seq_length)
    
    x_array = np.array(int_text[:num_batches*(batch_size*seq_length)])
    y_array = np.array(int_text[1:num_batches*(batch_size*seq_length)+1])
    # The last target of the last batch should be the first input of the first batch.
    y_array[-1]=x_array[0]
    
    x_batches = np.split(x_array.reshape(batch_size, -1), num_batches, 1)
    y_batches = np.split(y_array.reshape(batch_size, -1), num_batches, 1)
    
    output = np.array(list(zip(x_batches, y_batches)))
       
    return output

sample_output = get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2) 
print (sample_output)
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

[[[[ 1  2]
   [ 7  8]
   [13 14]]

  [[ 2  3]
   [ 8  9]
   [14 15]]]


 [[[ 3  4]
   [ 9 10]
   [15 16]]

  [[ 4  5]
   [10 11]
   [16 17]]]


 [[[ 5  6]
   [11 12]
   [17 18]]

  [[ 6  7]
   [12 13]
   [18  1]]]]
Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [14]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 32
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 16
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 10


#num_epochs, batch_size,  rnn_size, embed_dim, seq_length, learning_rate  train_loss
#100          32           512         256          16           0.01       0.790
#100          64           512         256          16           0.01       0.227
#100          128          512         256          16           0.01       0.214
#100          256          512         256          16           0.01       0.196,  0.233 
#50           256          512         256          16           0.01       0.486

#100          32           1024        256          16           0.01       1.404
#100          64           1024        256          16           0.01       0.228
#100          128          1024        256          16           0.01       0.543
#100          256          1024        256          16           0.01       0.552

#100          32           512         256          16           0.001      0.239
#100          64           512         256          16           0.001      0.225
#100          128          512         256          16           0.001      0.437
#100          256          512         256          16           0.001      2.564
#50           32           512         256          16           0.001      0.281
#50           64           512         256          16           0.001      0.710



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/134   train_loss = 8.821
Epoch   0 Batch   10/134   train_loss = 6.766
Epoch   0 Batch   20/134   train_loss = 6.702
Epoch   0 Batch   30/134   train_loss = 6.829
Epoch   0 Batch   40/134   train_loss = 6.881
Epoch   0 Batch   50/134   train_loss = 6.530
Epoch   0 Batch   60/134   train_loss = 6.415
Epoch   0 Batch   70/134   train_loss = 6.337
Epoch   0 Batch   80/134   train_loss = 6.332
Epoch   0 Batch   90/134   train_loss = 6.692
Epoch   0 Batch  100/134   train_loss = 6.544
Epoch   0 Batch  110/134   train_loss = 6.770
Epoch   0 Batch  120/134   train_loss = 6.489
Epoch   0 Batch  130/134   train_loss = 6.371
Epoch   1 Batch    6/134   train_loss = 6.214
Epoch   1 Batch   16/134   train_loss = 6.030
Epoch   1 Batch   26/134   train_loss = 6.108
Epoch   1 Batch   36/134   train_loss = 6.167
Epoch   1 Batch   46/134   train_loss = 6.331
Epoch   1 Batch   56/134   train_loss = 6.149
Epoch   1 Batch   66/134   train_loss = 5.845
Epoch   1 Batch   76/134   train_l

Epoch  13 Batch   48/134   train_loss = 3.798
Epoch  13 Batch   58/134   train_loss = 4.078
Epoch  13 Batch   68/134   train_loss = 3.822
Epoch  13 Batch   78/134   train_loss = 3.670
Epoch  13 Batch   88/134   train_loss = 3.877
Epoch  13 Batch   98/134   train_loss = 3.663
Epoch  13 Batch  108/134   train_loss = 3.636
Epoch  13 Batch  118/134   train_loss = 3.768
Epoch  13 Batch  128/134   train_loss = 3.763
Epoch  14 Batch    4/134   train_loss = 3.709
Epoch  14 Batch   14/134   train_loss = 3.852
Epoch  14 Batch   24/134   train_loss = 3.770
Epoch  14 Batch   34/134   train_loss = 3.822
Epoch  14 Batch   44/134   train_loss = 3.886
Epoch  14 Batch   54/134   train_loss = 3.693
Epoch  14 Batch   64/134   train_loss = 3.756
Epoch  14 Batch   74/134   train_loss = 3.664
Epoch  14 Batch   84/134   train_loss = 3.810
Epoch  14 Batch   94/134   train_loss = 3.822
Epoch  14 Batch  104/134   train_loss = 3.741
Epoch  14 Batch  114/134   train_loss = 3.702
Epoch  14 Batch  124/134   train_l

Epoch  26 Batch   96/134   train_loss = 2.163
Epoch  26 Batch  106/134   train_loss = 2.118
Epoch  26 Batch  116/134   train_loss = 2.106
Epoch  26 Batch  126/134   train_loss = 2.146
Epoch  27 Batch    2/134   train_loss = 1.974
Epoch  27 Batch   12/134   train_loss = 2.077
Epoch  27 Batch   22/134   train_loss = 2.157
Epoch  27 Batch   32/134   train_loss = 2.089
Epoch  27 Batch   42/134   train_loss = 1.941
Epoch  27 Batch   52/134   train_loss = 2.106
Epoch  27 Batch   62/134   train_loss = 1.997
Epoch  27 Batch   72/134   train_loss = 1.758
Epoch  27 Batch   82/134   train_loss = 1.890
Epoch  27 Batch   92/134   train_loss = 1.977
Epoch  27 Batch  102/134   train_loss = 2.061
Epoch  27 Batch  112/134   train_loss = 2.088
Epoch  27 Batch  122/134   train_loss = 1.748
Epoch  27 Batch  132/134   train_loss = 1.846
Epoch  28 Batch    8/134   train_loss = 2.087
Epoch  28 Batch   18/134   train_loss = 1.860
Epoch  28 Batch   28/134   train_loss = 1.932
Epoch  28 Batch   38/134   train_l

Epoch  40 Batch   10/134   train_loss = 0.654
Epoch  40 Batch   20/134   train_loss = 0.705
Epoch  40 Batch   30/134   train_loss = 0.621
Epoch  40 Batch   40/134   train_loss = 0.594
Epoch  40 Batch   50/134   train_loss = 0.576
Epoch  40 Batch   60/134   train_loss = 0.621
Epoch  40 Batch   70/134   train_loss = 0.612
Epoch  40 Batch   80/134   train_loss = 0.629
Epoch  40 Batch   90/134   train_loss = 0.664
Epoch  40 Batch  100/134   train_loss = 0.534
Epoch  40 Batch  110/134   train_loss = 0.612
Epoch  40 Batch  120/134   train_loss = 0.533
Epoch  40 Batch  130/134   train_loss = 0.609
Epoch  41 Batch    6/134   train_loss = 0.553
Epoch  41 Batch   16/134   train_loss = 0.544
Epoch  41 Batch   26/134   train_loss = 0.599
Epoch  41 Batch   36/134   train_loss = 0.513
Epoch  41 Batch   46/134   train_loss = 0.449
Epoch  41 Batch   56/134   train_loss = 0.520
Epoch  41 Batch   66/134   train_loss = 0.493
Epoch  41 Batch   76/134   train_loss = 0.499
Epoch  41 Batch   86/134   train_l

Epoch  53 Batch   58/134   train_loss = 0.257
Epoch  53 Batch   68/134   train_loss = 0.228
Epoch  53 Batch   78/134   train_loss = 0.236
Epoch  53 Batch   88/134   train_loss = 0.268
Epoch  53 Batch   98/134   train_loss = 0.307
Epoch  53 Batch  108/134   train_loss = 0.263
Epoch  53 Batch  118/134   train_loss = 0.289
Epoch  53 Batch  128/134   train_loss = 0.256
Epoch  54 Batch    4/134   train_loss = 0.250
Epoch  54 Batch   14/134   train_loss = 0.246
Epoch  54 Batch   24/134   train_loss = 0.319
Epoch  54 Batch   34/134   train_loss = 0.270
Epoch  54 Batch   44/134   train_loss = 0.224
Epoch  54 Batch   54/134   train_loss = 0.262
Epoch  54 Batch   64/134   train_loss = 0.226
Epoch  54 Batch   74/134   train_loss = 0.281
Epoch  54 Batch   84/134   train_loss = 0.302
Epoch  54 Batch   94/134   train_loss = 0.261
Epoch  54 Batch  104/134   train_loss = 0.253
Epoch  54 Batch  114/134   train_loss = 0.258
Epoch  54 Batch  124/134   train_loss = 0.269
Epoch  55 Batch    0/134   train_l

Epoch  66 Batch  106/134   train_loss = 0.217
Epoch  66 Batch  116/134   train_loss = 0.276
Epoch  66 Batch  126/134   train_loss = 0.267
Epoch  67 Batch    2/134   train_loss = 0.256
Epoch  67 Batch   12/134   train_loss = 0.262
Epoch  67 Batch   22/134   train_loss = 0.257
Epoch  67 Batch   32/134   train_loss = 0.250
Epoch  67 Batch   42/134   train_loss = 0.228
Epoch  67 Batch   52/134   train_loss = 0.246
Epoch  67 Batch   62/134   train_loss = 0.239
Epoch  67 Batch   72/134   train_loss = 0.223
Epoch  67 Batch   82/134   train_loss = 0.215
Epoch  67 Batch   92/134   train_loss = 0.262
Epoch  67 Batch  102/134   train_loss = 0.284
Epoch  67 Batch  112/134   train_loss = 0.274
Epoch  67 Batch  122/134   train_loss = 0.195
Epoch  67 Batch  132/134   train_loss = 0.214
Epoch  68 Batch    8/134   train_loss = 0.210
Epoch  68 Batch   18/134   train_loss = 0.233
Epoch  68 Batch   28/134   train_loss = 0.191
Epoch  68 Batch   38/134   train_loss = 0.232
Epoch  68 Batch   48/134   train_l

Epoch  80 Batch   20/134   train_loss = 0.275
Epoch  80 Batch   30/134   train_loss = 0.224
Epoch  80 Batch   40/134   train_loss = 0.232
Epoch  80 Batch   50/134   train_loss = 0.192
Epoch  80 Batch   60/134   train_loss = 0.238
Epoch  80 Batch   70/134   train_loss = 0.222
Epoch  80 Batch   80/134   train_loss = 0.207
Epoch  80 Batch   90/134   train_loss = 0.253
Epoch  80 Batch  100/134   train_loss = 0.197
Epoch  80 Batch  110/134   train_loss = 0.232
Epoch  80 Batch  120/134   train_loss = 0.179
Epoch  80 Batch  130/134   train_loss = 0.245
Epoch  81 Batch    6/134   train_loss = 0.224
Epoch  81 Batch   16/134   train_loss = 0.226
Epoch  81 Batch   26/134   train_loss = 0.265
Epoch  81 Batch   36/134   train_loss = 0.240
Epoch  81 Batch   46/134   train_loss = 0.188
Epoch  81 Batch   56/134   train_loss = 0.231
Epoch  81 Batch   66/134   train_loss = 0.220
Epoch  81 Batch   76/134   train_loss = 0.243
Epoch  81 Batch   86/134   train_loss = 0.171
Epoch  81 Batch   96/134   train_l

Epoch  93 Batch   68/134   train_loss = 0.192
Epoch  93 Batch   78/134   train_loss = 0.193
Epoch  93 Batch   88/134   train_loss = 0.235
Epoch  93 Batch   98/134   train_loss = 0.270
Epoch  93 Batch  108/134   train_loss = 0.230
Epoch  93 Batch  118/134   train_loss = 0.252
Epoch  93 Batch  128/134   train_loss = 0.218
Epoch  94 Batch    4/134   train_loss = 0.210
Epoch  94 Batch   14/134   train_loss = 0.207
Epoch  94 Batch   24/134   train_loss = 0.286
Epoch  94 Batch   34/134   train_loss = 0.241
Epoch  94 Batch   44/134   train_loss = 0.180
Epoch  94 Batch   54/134   train_loss = 0.219
Epoch  94 Batch   64/134   train_loss = 0.192
Epoch  94 Batch   74/134   train_loss = 0.242
Epoch  94 Batch   84/134   train_loss = 0.259
Epoch  94 Batch   94/134   train_loss = 0.221
Epoch  94 Batch  104/134   train_loss = 0.217
Epoch  94 Batch  114/134   train_loss = 0.227
Epoch  94 Batch  124/134   train_loss = 0.242
Epoch  95 Batch    0/134   train_loss = 0.200
Epoch  95 Batch   10/134   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [19]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_tensor = loaded_graph.get_tensor_by_name('input:0')
    initial_state_tensor = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state_tensor = loaded_graph.get_tensor_by_name('final_state:0')
    probs_tensor = loaded_graph.get_tensor_by_name('probs:0')
    return (input_tensor, initial_state_tensor, final_state_tensor, probs_tensor)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [20]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    next_word = np.random.choice(list(int_to_vocab.values()),p=probabilities)
    return next_word
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [21]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:... i didn't rip out his voice box, but i did stretch out.
chief_wiggum: uh, hello? we can get back in business... uh, i was gonna go homer in the stuff here.
marge_simpson: what?
homer_simpson:(grunt) what's the hubub book, boys.
moe_szyslak: yeah, i know, your dad's the two thing to do. and now, i know, this is notably thing.
carl_carlson:(worried) i'm the best of this than moe's. it, uh, good uh, let me check. but, i'm gonna do something with my partner, made in the air room. he might have better to buy him.
moe_szyslak: listen to dumb? oh, i ain't be mad at those all life.
lenny_leonard: what are you going?
homer_simpson: marge_simpson: when you're right, moe. don't as nothin'.
marge_simpson: to put. the answer.
marge_simpson:(sad) yeah, we all got any letter before...(smug laugh)



# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.